This notebook edits the sets `modelAE5_FP_2020_test.csv` and `modelAE5_FP_2020_train.csv` created on January 30 to be create new test/train sets for a glcm model. 

These modelAE5_FP train/test sets have points in different crss and also have some points that fall outside the coastal area. 

Changes made to the  modelAE5_FP_2020_test/train.csv:

* updated them so all ponts are in crs EPSG:4326
* deleted vegetation and iceplant points that fall outside coastal area (119 for training, X for test)


**notebook has to be on main folder to run**

In [1]:
import pandas as pd
from rasterio import CRS
import A_data_sampling_workflow.sample_rasters as sr
import os
import geopandas as gpd
import numpy as np

In [2]:
pts = pd.read_csv('/home/jovyan/msai4earth-esa/iceplant_detection/models/modelAE5_FP_2020/modelAE5_FP_2020_test.csv')

In [3]:
pts.groupby(['iceplant']).count()

,x,y,pts_crs,aoi,naip_id,r,r_max,r_min,r_diff,r_avg,...,g_corrN,g_corrE,b_contN,b_contE,g_corrN.1,b_corrE,nir_contN,nir_contE,nir_corrN,nir_corrE
iceplant,,,,,,,,,,,,,,,,,,,,,
0.0,558,558,558,558,558,558,558,558,558,558,...,558,558,558,558,558,558,558,558,558,558
1.0,230,230,230,230,230,230,230,230,230,230,...,230,230,230,230,230,230,230,230,230,230


In [4]:
# removed duplicated green glcm correlation N
pts = pts.drop(['g_corrN.1'],axis=1)
pts.columns

Index(['x', 'y', 'pts_crs', 'aoi', 'naip_id', 'r', 'r_max', 'r_min', 'r_diff',
       'r_avg', 'r_entr', 'g', 'g_max', 'g_min', 'g_diff', 'g_avg', 'g_entr',
       'b', 'b_max', 'b_min', 'b_diff', 'b_avg', 'b_entr', 'nir', 'nir_max',
       'nir_min', 'nir_diff', 'nir_avg', 'nir_entr', 'ndvi', 'ndvi_max',
       'ndvi_min', 'ndvi_diff', 'ndvi_avg', 'ndvi_entr', 'year', 'month',
       'day_in_year', 'r_entr5', 'g_entr5', 'b_entr5', 'nir_entr5',
       'ndvi_entr5', 'r_avg5', 'g_avg5', 'b_avg5', 'nir_avg5', 'r_contN',
       'r_contE', 'r_corrN', 'r_corrE', 'g_contN', 'g_contE', 'g_corrN',
       'g_corrE', 'b_contN', 'b_contE', 'b_corrE', 'nir_contN', 'nir_contE',
       'nir_corrN', 'nir_corrE', 'iceplant'],
      dtype='object')

In [5]:
crss = pts.pts_crs.unique()
common_crs = CRS.from_epsg(4326)

new_coords = []

for crs_str in crss:
    crs = CRS.from_string(crs_str)    
    pts_crs = pts[pts.pts_crs == crs_str]  

    pts_gdf = sr.geodataframe_from_csv(df = pts_crs, lon_label='x', lat_label='y', crs=crs)
    pts_col = pts_gdf.to_crs(common_crs).geometry

    new_coords.append(pd.DataFrame({'x':pts_col.x, 'y':pts_col.y}))

In [6]:
pts = pts.drop(['x','y','pts_crs'], axis=1)
new_xy = pd.concat(new_coords).sort_index()
new_pts = pd.concat([pts,new_xy],axis=1)
new_pts['pts_crs']='EPSG:4326'

# order columns
new_pts = new_pts[['x', 'y', 'pts_crs', 'aoi', 'naip_id', 'r', 'r_max', 'r_min', 'r_diff',
       'r_avg', 'r_entr', 'g', 'g_max', 'g_min', 'g_diff', 'g_avg', 'g_entr',
       'b', 'b_max', 'b_min', 'b_diff', 'b_avg', 'b_entr', 'nir', 'nir_max',
       'nir_min', 'nir_diff', 'nir_avg', 'nir_entr', 'ndvi', 'ndvi_max',
       'ndvi_min', 'ndvi_diff', 'ndvi_avg', 'ndvi_entr', 'year', 'month',
       'day_in_year', 'r_entr5', 'g_entr5', 'b_entr5', 'nir_entr5',
       'ndvi_entr5', 'r_avg5', 'g_avg5', 'b_avg5', 'nir_avg5', 'r_contN',
       'r_contE', 'r_corrN', 'r_corrE', 'g_contN', 'g_contE', 'g_corrN',
       'g_corrE', 'b_contN', 'b_contE', 'b_corrE', 'nir_contN', 'nir_contE',
       'nir_corrN', 'nir_corrE', 'iceplant']]

In [7]:
# only keep points within coastal zone
fp = os.path.join(os.getcwd(), 
                  'separating_naip_flights', 
                  'SB_coastal_buffer', 
                  'SB_coastal_buffer.shp')
coast = gpd.read_file(fp).to_crs(common_crs)
poly = coast.geometry[0]

In [8]:
only_pts = sr.geodataframe_from_csv(df = new_pts, lon_label='x', lat_label='y', crs=common_crs).geometry

In [9]:
new_pts['in_coast'] = [poly.contains(x) for x in only_pts]
new_pts = new_pts[new_pts.in_coast == True]
new_pts = new_pts.reset_index(drop=True)
new_pts

,x,y,pts_crs,aoi,naip_id,r,r_max,r_min,r_diff,r_avg,...,b_contN,b_contE,b_corrE,nir_contN,nir_contE,nir_corrN,nir_corrE,iceplant,geometry,in_coast
0,-120.492356,34.499585,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,114,129,106,23,118.777780,...,9.309091,17.709091,0.431129,40.054545,49.690909,0.838753,0.777462,0.0,POINT (-120.49236 34.49959),True
1,-120.490025,34.497770,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,115,124,106,18,115.666664,...,21.372727,12.118182,0.856829,79.272727,25.981818,0.738001,0.921704,0.0,POINT (-120.49002 34.49777),True
2,-120.486650,34.492685,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,105,121,99,22,109.444440,...,21.254545,30.509091,0.664311,39.236364,47.663636,0.712933,0.673046,0.0,POINT (-120.48665 34.49269),True
3,-120.482762,34.489505,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,90,106,84,22,93.777780,...,34.300000,79.236364,0.732288,53.745455,61.218182,0.816360,0.789773,0.0,POINT (-120.48276 34.48951),True
4,-120.482946,34.487116,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,129,139,122,17,130.888890,...,27.681818,38.527273,0.765100,21.300000,39.727273,0.886709,0.784582,0.0,POINT (-120.48295 34.48712),True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734,-120.437732,34.455862,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,101,121,100,21,106.666664,...,36.136364,187.518182,0.887230,37.000000,67.881818,0.898070,0.807652,0.0,POINT (-120.43773 34.45586),True
735,-120.443079,34.455301,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,121,131,102,29,118.888885,...,19.636364,22.545455,0.735986,16.463636,22.163636,0.893864,0.870406,0.0,POINT (-120.44308 34.45530),True
736,-120.445337,34.456349,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,136,147,118,29,134.444440,...,16.627273,30.354545,0.712966,10.845455,12.500000,0.899808,0.891552,0.0,POINT (-120.44534 34.45635),True
737,-120.445716,34.455899,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,126,142,114,28,129.111110,...,32.036364,33.936364,0.725589,13.345455,12.245455,0.826525,0.838465,0.0,POINT (-120.44572 34.45590),True


In [10]:
new_pts = new_pts.drop(['geometry','in_coast'], axis=1)

In [11]:
new_pts

,x,y,pts_crs,aoi,naip_id,r,r_max,r_min,r_diff,r_avg,...,g_corrN,g_corrE,b_contN,b_contE,b_corrE,nir_contN,nir_contE,nir_corrN,nir_corrE,iceplant
0,-120.492356,34.499585,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,114,129,106,23,118.777780,...,0.603009,0.380552,9.309091,17.709091,0.431129,40.054545,49.690909,0.838753,0.777462,0.0
1,-120.490025,34.497770,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,115,124,106,18,115.666664,...,0.761263,0.887066,21.372727,12.118182,0.856829,79.272727,25.981818,0.738001,0.921704,0.0
2,-120.486650,34.492685,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,105,121,99,22,109.444440,...,0.758445,0.719317,21.254545,30.509091,0.664311,39.236364,47.663636,0.712933,0.673046,0.0
3,-120.482762,34.489505,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,90,106,84,22,93.777780,...,0.843629,0.703210,34.300000,79.236364,0.732288,53.745455,61.218182,0.816360,0.789773,0.0
4,-120.482946,34.487116,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,129,139,122,17,130.888890,...,0.791886,0.707312,27.681818,38.527273,0.765100,21.300000,39.727273,0.886709,0.784582,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734,-120.437732,34.455862,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,101,121,100,21,106.666664,...,0.960082,0.865365,36.136364,187.518182,0.887230,37.000000,67.881818,0.898070,0.807652,0.0
735,-120.443079,34.455301,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,121,131,102,29,118.888885,...,0.762672,0.703916,19.636364,22.545455,0.735986,16.463636,22.163636,0.893864,0.870406,0.0
736,-120.445337,34.456349,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,136,147,118,29,134.444440,...,0.838361,0.580622,16.627273,30.354545,0.712966,10.845455,12.500000,0.899808,0.891552,0.0
737,-120.445716,34.455899,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,126,142,114,28,129.111110,...,0.529539,0.596444,32.036364,33.936364,0.725589,13.345455,12.245455,0.826525,0.838465,0.0


In [12]:
new_pts.groupby(['iceplant']).count()

,x,y,pts_crs,aoi,naip_id,r,r_max,r_min,r_diff,r_avg,...,g_contE,g_corrN,g_corrE,b_contN,b_contE,b_corrE,nir_contN,nir_contE,nir_corrN,nir_corrE
iceplant,,,,,,,,,,,,,,,,,,,,,
0.0,521,521,521,521,521,521,521,521,521,521,...,521,521,521,521,521,521,521,521,521,521
1.0,218,218,218,218,218,218,218,218,218,218,...,218,218,218,218,218,218,218,218,218,218


In [13]:
new_pts.to_csv('new_test.csv', index=False)